In [2]:
pip install protobuf==3.20

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.3
    Uninstalling protobuf-4.24.3:
      Successfully uninstalled protobuf-4.24.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
! pip install numpy==1.18.5

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
  Using cached http://mirrors.tencentyun.com/pypi/packages/01/c6/87592f924246da1e58673cf708a2748754517c5cf050726238d6cfbd8df4/numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.3 requires numpy<2,>=1.20, but you have numpy 1.18.5 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.18.5 which is incompatible.
pandas 2.0.3 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.18.5 which is incompatible.
scikit-image 0.21.0 requires numpy>=1.21.1, but you have numpy 1.18.5 which is incompatible.
scikit-image 0.21.0 requires scipy>=1.8, but y

# 0. aug test

In [92]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os


def augment_one_image(img_file_path, dst_dir, num):
    image_data_gen =  ImageDataGenerator(
        rotation_range=150,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range = 0.1,
        shear_range=0,
        horizontal_flip=True
    )

    p = img_file_path
    img = load_img( p )

    img_array = img_to_array(img)
    img_array = img_array.reshape( (1, img_array.shape[0], img_array.shape[1], img_array.shape[2]) )

    i = 0
    prefix='aug__'+os.path.basename(p).replace('.png','')
    for batch in image_data_gen.flow(img_array, batch_size=1,save_to_dir=dst_dir, save_prefix=prefix, save_format='png'):
        i = i+1
        if i>=num:
            break


# 1. no-padding-resize

In [93]:
import os

dist = {}
src = 'cell_data20/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '')
    if y not in dist:
        dist[y] = 0
    dist[y] += 1

In [94]:
dist

{'red_blood_cell': 2407,
 'schizont': 10,
 'trophozoite': 23,
 'gametocyte': 7,
 'ring': 1}

In [95]:
from shutil import copyfile
from tqdm import *


dst = 'cell_data23/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell':
        copyfile(fp_src,fp_dst)

100%|█████████████████████████████████████| 2448/2448 [00:00<00:00, 8966.88it/s]


In [96]:
# one process, too slow
from shutil import copyfile
from tqdm import *


dst = 'cell_data23/'

for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    fp_dst = os.path.join(dst, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell' or y=='difficult':
        continue
    else:
        num_now = dist[y]
        num = dist['red_blood_cell']//num_now
        augment_one_image(fp_src, dst, num)

100%|███████████████████████████████████████| 2448/2448 [04:59<00:00,  8.17it/s]


In [97]:
# check aug result
import os

dist = {}
src = 'cell_data23/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '').split('_')[0]
    if y not in dist:
        dist[y] = 0
    dist[y] += 1
    
dist

{'red': 2407,
 'schizont': 2374,
 'gametocyte': 2364,
 'trophozoite': 2381,
 'ring': 2130}

In [24]:
# multiple processes
from shutil import copyfile
from tqdm import *


dst = 'cell_data7/'
task_para_list = []


for d in tqdm(os.listdir(src)):
    fp_src = os.path.join(src, d)
    y = d.split('___')[2].replace('.png', '')
    if y=='red_blood_cell' or y=='difficult':
        continue
    else:
        num_now = dist[y]
        num = dist['red_blood_cell']//num_now
        task_para_list.append( (fp_src, dst, num) )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2918/2918 [00:00<00:00, 330078.46it/s]


In [22]:
len(task_para_list)

61

In [23]:
import time
from multiprocessing import Process
import os


thnum = 8

def runs(sub_task_para_list):
    for xxx in sub_task_para_list:
        fp_src, dst, num = xxx
        augment_one_image(fp_src, dst, num)
        
n = len(task_para_list)//thnum
for i in range(thnum):
    p = Process(target=runs, args=(task_para_list[i*n:(i+1)*n],))  # target进程执行的任务, args传参数（元祖）
    p.start() 

In [42]:
# check aug result
import os

dist = {}
src = 'cell_data6/'
for d in os.listdir(src):
    y = d.split('___')[2].replace('.png', '').split('_')[0]
    if y not in dist:
        dist[y] = 0
    dist[y] += 1
    
dist

{'ring': 2610, 'trophozoite': 2778, 'schizont': 2816, 'gametocyte': 2100}